In [1]:
minimum_precision = 4
# THIS IS AN INCLUSIVE RANGE!!
maximum_precision = 16

In [2]:
body = """
use crate::prelude::*;

#[inline]
pub fn {method_name}(words: &[u32; {number_of_words}]) -> (usize, f32) {{
    let mut raw_estimate = 0.0;
    let mut number_of_zero_registers = 0;

{operations}

    (
number_of_zero_registers,
raw_estimate
    )
}}
"""

dispatcher = """
use crate::prelude::*;

#[inline]
pub fn get_generic_approximated_cardinality<const N: usize, const PRECISION: usize, const NUMBER_OF_REGISTERS_IN_WORD: usize>(
    words: &[u32; N],
) -> (usize, f32) {{
    let number_of_registers: usize = 1 << PRECISION;
    let number_of_bits_per_register: usize = 32 / NUMBER_OF_REGISTERS_IN_WORD;
    let mask = (1 << number_of_bits_per_register) - 1;
    
    words
        .iter()
        .copied()
        .flat_map(|six_registers| {{
            (0..NUMBER_OF_REGISTERS_IN_WORD).map(move |i| {{
                six_registers >> i * number_of_bits_per_register & mask
            }})
        }})
        .take(number_of_registers)
        .fold((0, 0.0), |(number_of_zero_registers, raw_estimate), register|{{
            (
                number_of_zero_registers + (register == 0) as usize,
                raw_estimate + 1.0 / (1u64 << register) as f32,
            )
        }})
}}

#[inline]
pub fn dispatch_specialized_count<
    const N: usize,
    const PRECISION: usize,
    const NUMBER_OF_REGISTERS_IN_WORD: usize
>(
    words: &[u32; N],
) -> (usize, f32) {{
    match (N, PRECISION, NUMBER_OF_REGISTERS_IN_WORD) {{
{cases}
        _ => get_generic_approximated_cardinality::<N, PRECISION, NUMBER_OF_REGISTERS_IN_WORD>(words)
    }}
}}

"""

In [3]:
import shutil
from math import ceil
import os

path = "src/specialized_counts"

if os.path.exists(path):
    shutil.rmtree(path)
    
os.makedirs(path)

module = []
cases = []

for bits_per_register in (5, 6, 8):
    registry_per_word = 32 // bits_per_register
    for precision in range(minimum_precision, maximum_precision - 3):
        number_of_registers = 1 << precision
        number_of_words = int(ceil(number_of_registers / registry_per_word))

        assign_registers = []
        check_zeros = []
        harmonic_sum = []
        total_registry_number = 0
        for word_number in range(number_of_words):
            registers = [
                f"register_{registry_number}" if registry_number < number_of_registers else "_"
                for registry_number in range(
                    total_registry_number,
                    total_registry_number + registry_per_word
                )
            ]
            
            registers_array = ", ".join(registers)
            estimation = " + ".join([
                f"1.0 / (1u64 << {register}) as f32"
                for register in registers
                if register != "_"
            ])
            zeros = " + ".join([
                f"({register} == 0) as usize"
                for register in registers
                if register != "_"
            ])
            assign_registers.append(
                f"\tlet [{registers_array}] = split_registers::<{registry_per_word}>(words[{word_number}]);\n"
                f"\traw_estimate += {estimation};\n"
                f"\tnumber_of_zero_registers += {zeros};"
            )
            total_registry_number += registry_per_word

        method_name = f"get_approximated_cardinality_with_{number_of_registers}_registers_and_{bits_per_register}_bits"

        completed_body = body.format(
            number_of_registers=number_of_registers,
            number_of_words=number_of_words,
            method_name=method_name,
            operations="\n".join(assign_registers),
        )

        cases.append(
            f"\t\t({number_of_words}, {precision}, {registry_per_word}) => {method_name}(unsafe {{ core::mem::transmute(words) }}),"
        )
        module.append(f"pub mod {method_name};")
        module.append(f"pub use {method_name}::*;")

        with open(f"{path}/{method_name}.rs", "w") as f:
            f.write(completed_body)
        
module.append("pub mod specialized_counts_dispatcher;")
module.append("pub use specialized_counts_dispatcher::*;")

with open(f"{path}/specialized_counts_dispatcher.rs", "w") as f:
    f.write(dispatcher.format(cases="\n".join(cases)))

with open(f"{path}/mod.rs", "w") as f:
    f.write("\n".join(module))
    